In [4]:
#!/usr/bin/python3
 
import pymysql
 
# 打开数据库连接
db = pymysql.connect("127.0.0.1","root","mysecret1997","mygopen")
 
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
 
# 使用 execute()  方法执行 SQL 查询 
cursor.execute("SELECT VERSION()")
 
# 使用 fetchone() 方法获取单条数据.
data = cursor.fetchone()
 
print ("Database version : %s " % data)
 
# 关闭数据库连接
db.close()

Database version : 8.0.17 


In [65]:
#!/usr/bin/python3
 
import pymysql
 
# 打开数据库连接
db = pymysql.connect("127.0.0.1","root","mysecret1997","mygopen")
 
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()

# 使用 execute() 方法执行 SQL，如果表存在则删除
cursor.execute("DROP TABLE IF EXISTS chrome_headless")
 
# 使用预处理语句创建表
sql = """CREATE TABLE chrome_headless (
         title  TEXT,
         href  TEXT)"""
 
cursor.execute(sql)

0

In [66]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import csv
################chrome_headless######################
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
#####################################################
get_title_and_href_dict = {}
for i in range(1,4,1):
    url = 'https://www.mygopen.com/search/label/%E8%AC%A0%E8%A8%80#archive-page-'+str(i)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    driver.refresh()
    WebDriverWait(driver,10).until(lambda d: d.find_elements_by_css_selector('.item-content .item-title a'))
    a_tag = driver.find_elements_by_css_selector('.item-content .item-title a')
    for j in range(len(a_tag)):
        get_title_and_href_dict.update({'title':a_tag[j].text,'href':a_tag[j].get_attribute('href')})

        #表名
        table = "chrome_headless"
        # 列的字段
        keys = ', '.join(get_title_and_href_dict.keys())
        # 行字段
        values = ', '.join(['%s']*len(get_title_and_href_dict))
        sql = 'INSERT INTO {table}({keys}) VALUES ({values})'.format(table=table, keys=keys, values=values)
        # 将字段的value转化为元祖存入
        cursor.execute(sql, tuple(get_title_and_href_dict.values()))
        db.commit()
        
try:
   # 执行sql语句
   cursor.execute(sql)
   # 提交到数据库执行
   db.commit()
except:
   # 如果发生错误则回滚
   db.rollback()

# 关闭数据库连接
db.close()